In [1]:
import pandas as pd
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
from transformers import set_seed
import tensorflow as tf
from tqdm import tqdm

/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = pd.read_csv("../data/train.csv")
val_df = pd.read_csv("../data/validation.csv")
test_df = pd.read_csv("../data/test.csv")

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [4]:
def tokenize(sentences, max_length=100, padding='max_length'):
    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf" 
    )

In [5]:
bert_x_train = train_df["Comment_Adj"].tolist()
bert_y_train = train_df["Result_Bin"].tolist()
bert_x_val = val_df["Comment_Adj"].tolist()
bert_y_val = val_df["Result_Bin"].tolist()

In [6]:
train_encodings = tokenize(bert_x_train)
val_encodings = tokenize(bert_x_val)

In [7]:
train_labels = tf.convert_to_tensor(bert_y_train, dtype=tf.int32)
val_labels = tf.convert_to_tensor(bert_y_val, dtype=tf.int32)

In [9]:
seed_value = 42
set_seed(seed_value)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),  
    train_labels
)).shuffle(1000).batch(30).prefetch(1)

validation_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),  
    val_labels
)).batch(30).prefetch(1)


### Without Custom Classifier

In [15]:
seed_value = 42
set_seed(seed_value)

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ["accuracy")

model.fit(
    x=train_dataset,
    y=None,
    validation_data=validation_dataset,
    batch_size=30,
    epochs=2
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/2
228/228 [==============================] - 976s 4s/step - loss: 0.6641 - accuracy: 0.5987 - val_loss: 0.6537 - val_accuracy: 0.5980
Epoch 2/2
228/228 [==============================] - 1016s 4s/step - loss: 0.5443 - accuracy: 0.7240 - val_loss: 0.6240 - val_accuracy: 0.6652


In [16]:
bert_x_test = test_df["Comment_Adj"].tolist()
bert_y_test = test_df["Result_Bin"].tolist()

test_encodings = tokenize(bert_x_test)

test_labels = tf.convert_to_tensor(bert_y_test, dtype=tf.int32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),  
    test_labels
)).shuffle(1000).batch(30).prefetch(1)

results = model.evaluate(test_dataset)
print("Loss:", results[0])
print("Accuracy:", results[1])

2024-04-28 13:32:35.788501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [2276,100]
	 [[{{node Placeholder/_1}}]]


76/76 [==============================] - 93s 1s/step - loss: 0.6230 - accuracy: 0.6586
Loss: 0.622992217540741
Accuracy: 0.6586115956306458


### With Custom Classifier

In [31]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, TFDistilBertModel, DistilBertConfig

seed_value = 42
set_seed(seed_value)

def tokenize(sentences, max_length=100, padding='max_length'):
    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf" 
    )

bert_x_train = train_df["Comment_Adj"].tolist()
bert_y_train = train_df["Result_Bin"].tolist()
bert_x_val = val_df["Comment_Adj"].tolist()
bert_y_val = val_df["Result_Bin"].tolist()

train_encodings = tokenize(bert_x_train)
val_encodings = tokenize(bert_x_val)

train_labels = tf.convert_to_tensor(bert_y_train, dtype=tf.int32)
val_labels = tf.convert_to_tensor(bert_y_val, dtype=tf.int32)


train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),  
    train_labels
)).shuffle(1000).batch(30).prefetch(1)

validation_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),  
    val_labels
)).batch(30).prefetch(1)

class CustomBertClassification(TFDistilBertForSequenceClassification):
    def __init__(self, config, *args, **kwargs):
        super(CustomBertClassification, self).__init__(config, *args, **kwargs)
        self.distilbert = TFDistilBertModel(config)
        self.pre_classifier = tf.keras.layers.Dense(768, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(config.num_labels, activation='softmax')

    def call(self, inputs, training=False, **kwargs):
        outputs = self.distilbert(inputs, **kwargs)
        hidden_state = outputs[0]  # Get the last hidden state
        pooled_output = hidden_state[:, 0]  # Use the first token for classification
        pooled_output = self.pre_classifier(pooled_output)
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return {'logits': logits} if not training else logits

configuration = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=2)
model_1 = CustomBertClassification(configuration)

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5)
model_1.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'] 
)

model_1.fit(
    x=train_dataset, 
    validation_data=validation_dataset,  
    batch_size=30,
    epochs=2
)

Epoch 1/2


2024-04-28 14:24:07.122147: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int32 and shape [6826]
	 [[{{node Placeholder/_2}}]]


AttributeError: in user code:

    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 1670, in train_step
        if list(y_pred.keys())[0] == "loss":

    AttributeError: 'Tensor' object has no attribute 'keys'


Epoch 1/2


AttributeError: in user code:

    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 1670, in train_step
        if list(y_pred.keys())[0] == "loss":

    AttributeError: 'Tensor' object has no attribute 'keys'
